In [38]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib as mpl
import re
from fastai.tabular import *

In [64]:
df_m = pd.read_csv("2007-2020.csv", header=0)
df_m.Place = df_m.Place.str.lower()
df_m.head() #(7680, 3)

,Date,Place,Ill
0,2010-12-16,dolnośląskie,1528
1,2010-12-08,dolnośląskie,2222
2,2010-12-01,dolnośląskie,2082
3,2010-11-23,dolnośląskie,2230
4,2010-11-16,dolnośląskie,1795


In [65]:
df_t = pd.read_csv("trends.csv", header=0, names=["Token", "Place", "Date", "Value"], parse_dates=['Date'])
df_trends_grypa = df_t[df_t.Token=="grypa"]
df_trends_kaszel = df_t[df_t.Token=="kaszel"]
df_trends_grypa.head()

,Token,Place,Date,Value
0,grypa,śląskie,2010-01-10,699.486201
1,grypa,śląskie,2010-01-17,782.967092
2,grypa,śląskie,2010-01-24,689.975973
3,grypa,śląskie,2010-01-31,773.999028
4,grypa,śląskie,2010-02-07,600.234477


In [75]:
# df_m.to_csv("Meld_trends.csv", index=False)
df_m = pd.read_csv("Table.csv", header=0, parse_dates=['Date'])
df_m.head()

,Date,Place,Ill,Trends_grypa,TempDob,TempDob_7D,Trends_kaszel,Wiatr
0,2010-01-01,podkarpackie,389,701.532723,0.469231,-0.039560,0.909890,0
1,2010-01-01,lubelskie,128,0.000000,-2.366667,-1.056522,0.937681,0
2,2010-01-01,podlaskie,240,3411.804845,-5.460000,-2.457143,0.808571,0
3,2010-01-01,łódzkie,2245,537.367082,-2.650000,-0.671429,1.064286,0
4,2010-01-01,wielkopolskie,1410,1437.400251,-2.583333,-0.405952,1.935714,0


In [5]:
########ADD TRENDS_GRYPA
items_t = [i for i in df_trends_grypa.Date]

def func_t(date, woj, items):
    date = pd.to_datetime(date)
    data_trends = min(items, key=lambda x: abs(x - date)) #najbliższa data --> zbierz z niej wartości
    data_trends = str(data_trends)[0:10]
    try:
        value_trends = df_trends_grypa.Value[(df_trends_grypa.Date == data_trends)&
                                         (df_trends_grypa.Place==woj)].values[0]
    except:
        value_trends = 0
    return value_trends

In [73]:
df_m.head()

,Date,Place,Ill,Trends_grypa,TempDob,TempDob_7D,Trends_kaszel,Wiatr
0,2010-01-01,podkarpackie,389,701.532723,0.469231,-0.039560,0.909890,0
1,2010-01-01,lubelskie,128,0.000000,-2.366667,-1.056522,0.937681,0
2,2010-01-01,podlaskie,240,3411.804845,-5.460000,-2.457143,0.808571,0
3,2010-01-01,łódzkie,2245,537.367082,-2.650000,-0.671429,1.064286,0
4,2010-01-01,wielkopolskie,1410,1437.400251,-2.583333,-0.405952,1.935714,0


In [30]:
########ADD TRENDS_KASZEL
items_t = [i for i in df_trends_kaszel.Date]

def func_t_k(date, woj, items):
    date = pd.to_datetime(date)
    data_trends = min(items, key=lambda x: abs(x - date)) #najbliższa data --> zbierz z niej wartości
    data_trends = str(data_trends)[0:10]
    try:
        value_trends = df_trends_kaszel.Value[(df_trends_kaszel.Date == data_trends)&
                                         (df_trends_kaszel.Place==woj)].values[0]
    except:
        value_trends = 0
    return value_trends

In [25]:
df_k = pd.read_csv("Klimat_woj.csv", header=0, parse_dates=['Date'])
df_k.head()

,Date,Kod stacji,Nazwa stacji,Średnia dobowa temperatura,Status pomiaru TEMP,Średnia dobowa wilgotność względna,Status pomiaru WLGS,Średnia dobowa prędkość wiatru,Status pomiaru FWS,Średnie dobowe zachmurzenie ogólne,Status pomiaru NOS,Wojewodztwo
0,2009-10-01,254170140,kościerzyna,7.3,NaN,90.0,NaN,1.7,NaN,5.0,NaN,pomorskie
1,2009-10-01,249200910,morskie oko,6.9,NaN,0.0,8.0,6.0,NaN,0.0,8.0,małopolskie
2,2009-10-01,253220070,biebrza-pieńczykówek,8.0,NaN,89.8,NaN,4.3,NaN,6.3,NaN,podlaskie
3,2009-10-01,249190560,jabłonka,8.9,NaN,0.0,8.0,1.7,NaN,6.3,NaN,małopolskie
4,2009-10-01,249220280,stuposiany,6.5,NaN,88.0,NaN,2.0,NaN,3.7,NaN,podkarpackie


In [128]:
df_o = pd.read_csv("Opad_woj.csv", header=0, parse_dates=['Date'])
df_o.head()

,Date,Kod stacji,Nazwa stacji,Suma dobowa opadów,Status pomiaru,Rodzaj opadu,Wysokość pokrywy śnieżnej,Status pomiaru PKSN,Wysokość świeżo spadłego śniegu,Status pomiaru HSS,Gatunek śniegu,Status pomiaru GATS,Rodzaj pokrywy śnieżnej,Status pomiaru RPSN,Wojewodztwo
0,2009-10-01,251180040,poddębice,4.9,NaN,W,0,8.0,0,8.0,NaN,8.0,NaN,8.0,łódzkie
1,2009-10-01,251180030,dobra,2.6,NaN,W,0,8.0,0,8.0,NaN,8.0,NaN,8.0,zachodniopomorskie
2,2009-10-01,251180010,uniejów,6.8,NaN,W,0,8.0,0,8.0,NaN,8.0,NaN,8.0,łódzkie
3,2009-10-01,251170310,włochy,7.3,NaN,W,0,8.0,0,8.0,NaN,8.0,NaN,8.0,mazowieckie
4,2009-10-01,251170270,bierutów,4.6,NaN,W,0,8.0,0,8.0,NaN,8.0,NaN,8.0,dolnośląskie


In [129]:
df_s = pd.read_csv("Syno_woj.csv", header=0, parse_dates=['Date'])
df_s.head()

,Date,Kod stacji,Nazwa stacji,Średnie dobowe zachmurzenie ogólne [oktanty],Status pomiaru NOS,Średnia dobowa prędkość wiatru [m/s],Status pomiaru FWS,Średnia dobowa temperatura [°C],Status pomiaru TEMP,Średnia dobowe ciśnienie pary wodnej [hPa],...,Status pomiaru WLGS,Średnia dobowe ciśnienie na poziomie stacji [hPa],Status pomiaru PPPS,Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień,Status pomiaru WODZ,Suma opadu noc,Status pomiaru WONO,Wojewodztwo
0,2009-10-01,353210280,mikołajki,5.3,NaN,4.9,NaN,7.8,NaN,9.1,...,NaN,986.5,NaN,1001.9,NaN,2.2,NaN,2.7,NaN,warmińsko-mazurskie
1,2009-10-01,351230497,włodawa,5.8,NaN,5.4,NaN,8.5,NaN,10.1,...,NaN,986.0,NaN,1007.6,NaN,2.5,NaN,0.0,NaN,lubelskie
2,2009-10-01,349190625,zakopane,5.1,NaN,1.8,NaN,9.8,NaN,9.0,...,NaN,913.2,NaN,750.4,NaN,0.0,NaN,4.3,NaN,małopolskie
3,2009-10-01,351160415,legnica,6.1,NaN,5.5,NaN,12.9,NaN,12.3,...,NaN,994.0,NaN,1008.9,NaN,1.8,NaN,0.0,NaN,dolnośląskie
4,2009-10-01,351150400,zielona góra,6.6,NaN,4.8,NaN,11.2,NaN,11.4,...,NaN,985.1,NaN,1008.1,NaN,2.4,NaN,0.0,NaN,lubuskie


In [149]:
df_s.iloc[1:2, 5:25]

,Średnia dobowa prędkość wiatru [m/s],Status pomiaru FWS,Średnia dobowa temperatura [°C],Status pomiaru TEMP,Średnia dobowe ciśnienie pary wodnej [hPa],Status pomiaru CPW,Średnia dobowa wilgotność względna [%],Status pomiaru WLGS,Średnia dobowe ciśnienie na poziomie stacji [hPa],Status pomiaru PPPS,Średnie dobowe ciśnienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzień,Status pomiaru WODZ,Suma opadu noc,Status pomiaru WONO,Wojewodztwo
1,5.4,NaN,8.5,NaN,10.1,NaN,88.4,NaN,986.0,NaN,1007.6,NaN,2.5,NaN,0.0,NaN,lubelskie


In [74]:
df_m['TempDob'] = [0 for i in range(len(df_m))]
df_m['TempDob_7D'] = [0 for i in range(len(df_m))]
df_m['ZachmDob'] = [0 for i in range(len(df_m))]
df_m['Wiatr'] = [0 for i in range(len(df_m))]
df_m['Opad'] = [0 for i in range(len(df_m))]
df_m['DobZachm'] = [0 for i in range(len(df_m))]
df_m['Wiglot'] = [0 for i in range(len(df_m))]
df_m['Opad_7D'] = [0 for i in range(len(df_m))]

df_m.head()

,Date,Place,Ill,Trends_grypa,TempDob,TempDob_7D,Trends_kaszel,Wiatr,ZachmDob
0,2010-01-01,podkarpackie,389,701.532723,0,0,0.909890,0,0
1,2010-01-01,lubelskie,128,0.000000,0,0,0.937681,0,0
2,2010-01-01,podlaskie,240,3411.804845,0,0,0.808571,0,0
3,2010-01-01,łódzkie,2245,537.367082,0,0,1.064286,0,0
4,2010-01-01,wielkopolskie,1410,1437.400251,0,0,1.935714,0,0


In [84]:
df_m.iloc[0:1,3:4]

,Trends_grypa
0,701.532723


In [ ]:
#Trends grypa
items_t = [i for i in df_trends_grypa.Date]

for i in range(len(df_m)):
    date = pd.to_datetime(df_m.iloc[i:i+1,0:1].values[0][0])
    data_trends = min(items_t, key=lambda x: abs(x - date)) #najbliższa data --> zbierz z niej wartości
    data_trends = str(data_trends)[0:10]
    df_m.iloc[i:i+1,3:4] = df_trends_grypa.Value[(df_trends_grypa.Date == data_trends)&
                            (df_trends_grypa.Place==df_m.iloc[i:i+1, 1:2].values[0][0])].values[0]
    print(i)
    

In [83]:
#Trends Kaszel
items_t = [i for i in df_trends_kaszel.Date]

for i in range(len(df_m)):
    date = pd.to_datetime(df_m.iloc[i:i+1,0:1].values[0][0])
    data_trends = min(items_t, key=lambda x: abs(x - date)) #najbliższa data --> zbierz z niej wartości
    data_trends = str(data_trends)[0:10]
    df_m.iloc[i:i+1,6:7] = df_trends_kaszel.Value[(df_trends_kaszel.Date == data_trends)&
                            (df_trends_kaszel.Place==df_m.iloc[i:i+1, 1:2].values[0][0])].values[0] 
    

In [ ]:
#TempDob
for i in range(len(df_m)):
    df_m.iloc[i:i+1,4:5] =  np.mean(df_k['Średnia dobowa temperatura']
      [(df_k.Date==df_m.iloc[i:i+1, 0:1].values[0][0])&
       (df_k.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])
    
df_m.head()

In [ ]:
#TempDob_7D
for i in range(len(df_m)):
    date = pd.to_datetime(df_m.iloc[i:i+1,0:1].values[0][0])
    ranges = [date-timedelta(i) for i in range(0,7)]
    ranges = [str(ranges[i])[0:10] for i in range(0,7)]
    df_m.iloc[i:i+1,5:6] = np.mean(df_k['Średnia dobowa temperatura'][(df_k.Date.isin(ranges))&
       (df_k.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])

In [158]:
df_m.iloc[i:i+1, 9:10]

,DobZachm
7695,NaN


In [96]:
#Wiatr-Dob
for i in range(len(df_m)):    
    df_m.iloc[i:i+1,7:8] = np.mean(df_k['Średnia dobowa prędkość wiatru']
    [(df_k.Date==df_m.iloc[i:i+1,0:1].values[0][0])
    &(df_k.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])

In [137]:
#Opad
for i in range(len(df_m)):
    df_m.iloc[i:i+1,8:9] = np.mean(df_o['Suma dobowa opadów']
    [(df_o.Date==df_m.iloc[i:i+1,0:1].values[0][0])
    &(df_o.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])

In [143]:
#Średnie dobowe zachmurzenie ogólne
for i in range(len(df_m)):
    df_m.iloc[i:i+1,9:10] = np.mean(df_s['Średnie dobowe zachmurzenie ogólne [oktanty]']
    [(df_s.Date==df_m.iloc[i:i+1,0:1].values[0][0])
    &(df_s.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])

In [160]:
#Wilgot
for i in range(len(df_m)):
    df_m.iloc[i:i+1,10:11] = np.mean(df_s['Średnia dobowa wilgotność względna [%]']
    [(df_s.Date==df_m.iloc[i:i+1,0:1].values[0][0])
    &(df_s.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])

In [166]:
#Opad 7D
for i in range(len(df_m)):
    date = pd.to_datetime(df_m.iloc[i:i+1,0:1].values[0][0])
    ranges = [date-timedelta(i) for i in range(0,7)]
    ranges = [str(ranges[i])[0:10] for i in range(0,7)]
    
    df_m.iloc[i:i+1,11:12] = np.mean(df_o['Suma dobowa opadów']
    [(df_o.Date.isin(ranges))&(df_o.Wojewodztwo==df_m.iloc[i:i+1, 1:2].values[0][0])])

In [169]:
df_m.head()

,Date,Place,Ill,Trends_grypa,TempDob,TempDob_7D,Trends_kaszel,Wiatr,Opad,DobZachm,Wiglot,Opad_7D
0,2010-01-01,podkarpackie,389,701.532723,0.469231,-0.039560,701.532723,0.515385,1.145000,7.55,99.800000,2.775652
1,2010-01-01,lubelskie,128,0.000000,-2.366667,-1.056522,0.000000,0.888889,3.213636,8.00,93.200000,3.532061
2,2010-01-01,podlaskie,240,3411.804845,-5.460000,-2.457143,3411.804845,1.260000,5.995833,7.85,96.350000,4.768966
3,2010-01-01,łódzkie,2245,537.367082,-2.650000,-0.671429,1358.776160,0.575000,1.142857,8.00,94.766667,3.531000
4,2010-01-01,wielkopolskie,1410,1437.400251,-2.583333,-0.405952,928.425917,1.200000,0.960000,7.96,86.940000,2.269417


In [170]:
df_m.to_csv("Table.csv", index=False)